# [E-14]Movie Recommendation System ( 영화 추천 시스템 )
_____

## 목차
### 1. 개요
    1.1 들어가기에 앞서
    1.2 루브릭 평가기준
 
### 2. 프로젝트 : Movielens 영화 추천 실습
    2.1 데이터 준비 및 전처리
    2.2 데이터 분석
    2.3 선호하는 영화 ratings에 추가
    2.4 CSR matrix
    2.5 모델 훈련
    2.6 훈련 모델 예측 선호도 파악
    2.7 좋아하는 영화와 비슷한 영화 추천
    2.8 가장 좋아할 만한 영화 추천
### 3. 결론
    3.1 회고
    
-----

## 1. 개요

### 1.1 들어가기에 앞서

#### 학습 목표

- 추천 시스템의 개념과 목적을 이해한다.
- Implicit 라이브러리를 활용하여 Matrix Factorization(이하 MF) 기반의 추천 모델을 만들어 본다.
- 음악 감상 기록을 활용하여 비슷한 아티스트를 찾고 아티스트를 추천해 본다.
- 추천 시스템에서 자주 사용되는 데이터 구조인 CSR Matrix을 익힌다
- 유저의 행위 데이터 중 Explicit data와 Implicit data의 차이점을 익힌다.
- 새로운 데이터셋으로 직접 추천 모델을 만들어 본다.

---

### 1.2 루브릭 평가기준

평가문항|상세기준
-|-
1. CSR matrix가 정상적으로 만들어졌다.|사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.
2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.|사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.
3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.|MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.

---

## 2. 프로젝트 : Movielens 영화 추천 실습

### 2.1 데이터 준비 및 전처리

In [1]:
import os
import pandas as pd

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'count'}, inplace=True)

In [4]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
ratings = ratings.join(movies.set_index('movie_id'), on='movie_id')
ratings.head()

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


---

### 2.2 데이터 분석
- ratings에 있는 유니크한 영화 개수
- ratings에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [7]:
print('ratings에 있는 유니크한 영화 개수:' , ratings['movie_id'].nunique())

ratings에 있는 유니크한 영화 개수: 3628


In [8]:
print('ratings에 있는 유니크한 사용자 수:' , ratings['user_id'].nunique())

ratings에 있는 유니크한 사용자 수: 6039


In [9]:
movie_count = ratings.groupby('title')['movie_id'].count()
print('가장 인기 있는 영화 30개,' , movie_count.sort_values(ascending=False).head(30))

가장 인기 있는 영화 30개, title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                        

---

### 2.3 선호하는 영화 ratings에 추가

In [10]:
# 내가 선호하는 영화 5가지 추가
my_favorite = ['Back to the Future (1985)','Star Wars: Episode IV - A New Hope (1977)','Matrix, The (1999)','Toy Story (1995)','Men in Black (1997)']

my_movie = pd.DataFrame({'user_id': ['ko']*5, 'title': my_favorite, 'count': [5]*5})

if not ratings.isin({'user_id':['ko']})['user_id'].any():  
    ratings = ratings.append(my_movie)                         

ratings.tail(10)      

,user_id,movie_id,count,timestamp,title,genre
1000203,6040,1090.0,3,956715518.0,Platoon (1986),Drama|War
1000205,6040,1094.0,5,956704887.0,"Crying Game, The (1992)",Drama|Romance|War
1000206,6040,562.0,5,956704746.0,Welcome to the Dollhouse (1995),Comedy|Drama
1000207,6040,1096.0,4,956715648.0,Sophie's Choice (1982),Drama
1000208,6040,1097.0,4,956715569.0,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
0,ko,NaN,5,NaN,Back to the Future (1985),NaN
1,ko,NaN,5,NaN,Star Wars: Episode IV - A New Hope (1977),NaN
2,ko,NaN,5,NaN,"Matrix, The (1999)",NaN
3,ko,NaN,5,NaN,Toy Story (1995),NaN
4,ko,NaN,5,NaN,Men in Black (1997),NaN


---

### 2.4 CSR matrix

In [11]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['title'].unique()

# 유저, 영화 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [12]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings): # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data # ratings['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 movie 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie column indexing OK!!')
    ratings['title'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

ratings

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,count,timestamp,title,genre
0,0,1193.0,5,978300760.0,0,Drama
1,0,661.0,3,978302109.0,1,Animation|Children's|Musical
2,0,914.0,3,978301968.0,2,Musical|Romance
3,0,3408.0,4,978300275.0,3,Drama
4,0,2355.0,5,978824291.0,4,Animation|Children's|Comedy
...,...,...,...,...,...,...
0,6039,NaN,5,NaN,22,NaN
1,6039,NaN,5,NaN,44,NaN
2,6039,NaN,5,NaN,124,NaN
3,6039,NaN,5,NaN,40,NaN


In [13]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['title'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

---

### 2.5 모델 훈련

In [14]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [15]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [16]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [17]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

---

### 2.6 훈련 모델 예측 선호도 파악

In [18]:
ko, MeninBlack = user_to_idx['ko'], movie_to_idx['Men in Black (1997)']
ko_vector, MeninBlack_vector = als_model.user_factors[ko], als_model.item_factors[MeninBlack]

In [19]:
ko_vector

array([ 0.45921877, -0.30240497, -0.59310687,  0.21959107, -0.6245374 ,
       -0.3372988 , -0.20566013,  0.41829735,  0.05237757,  0.09820794,
       -0.69875693,  0.40466577, -0.367896  ,  0.23035811,  0.0984997 ,
       -0.88648236, -0.29833612,  1.1043415 , -0.66508126, -0.58340555,
       -0.5484292 ,  0.14986451,  0.15036099, -0.22534369, -0.7592236 ,
       -0.6150386 , -0.37434933,  1.4717165 , -0.7448279 ,  0.30980304,
        1.0818845 , -0.46234187, -0.6855248 ,  0.1624192 , -0.8706486 ,
       -0.41839656, -0.6227432 , -0.16861048,  0.6227385 , -0.20289762,
       -0.3819988 ,  0.18903935, -0.4712854 , -0.5205131 ,  0.20798653,
        0.40405145, -0.2779579 , -0.38001984,  0.05329349,  0.6839678 ,
        0.1265085 , -0.24700598, -0.21884325,  0.6078734 ,  0.57897633,
       -0.2456272 ,  0.83956516,  0.28668764,  0.09068739, -0.15607758,
       -0.6582305 , -0.18805815, -0.2251843 , -0.19878334,  0.45781893,
        0.65187025, -0.17733791, -0.36454085,  0.5149252 ,  0.71

In [20]:
MeninBlack_vector

array([-0.0196028 , -0.00202192, -0.00154387, -0.01349692,  0.00449033,
       -0.0070859 , -0.0090687 ,  0.00158368, -0.01088107,  0.00649552,
       -0.00252263,  0.02439101, -0.01211689,  0.03195802,  0.02887593,
       -0.02249205,  0.01716151,  0.02424812, -0.01305749, -0.03152207,
        0.01694363,  0.01888449,  0.02096028,  0.00500338, -0.00557387,
       -0.01189309,  0.02350542,  0.0370496 ,  0.01127739,  0.01957458,
        0.0576139 , -0.02800113, -0.01304858,  0.02841585, -0.01436681,
       -0.00950512, -0.02465231, -0.00165242,  0.00927619,  0.002062  ,
        0.00632879,  0.02027165,  0.02764341, -0.00843192, -0.00767748,
        0.00976668,  0.02802345, -0.01057083,  0.03644473, -0.00404717,
        0.01519912, -0.0117071 , -0.0180267 ,  0.03202694,  0.03662358,
       -0.01230396,  0.0214807 , -0.00309398, -0.01319056,  0.00931618,
       -0.01572387,  0.00948958, -0.01345964,  0.00392986,  0.01618493,
        0.04841513,  0.00444891, -0.00576006,  0.00158395,  0.02

In [21]:
# ko와 Star_Was를 내적하는 코드
np.dot(ko_vector, MeninBlack_vector)

0.5406644

In [22]:
Fargo = movie_to_idx['Fargo (1996)']
Fargo_vector = als_model.item_factors[Fargo]
np.dot(ko_vector, Fargo_vector)

0.04050297

---

### 2.7 좋아하는 영화와 비슷한 영화 추천

In [23]:
favorite_movie = 'Men in Black (1997)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(175, 1.0000002),
 (107, 0.84290457),
 (92, 0.6361099),
 (62, 0.5968012),
 (124, 0.48516345),
 (145, 0.47120306),
 (150, 0.4638064),
 (82, 0.44437888),
 (375, 0.4334118),
 (542, 0.42934904),
 (3466, 0.4279059),
 (670, 0.40846616),
 (320, 0.39245856),
 (138, 0.39071313),
 (87, 0.36646843)]

In [24]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Men in Black (1997)',
 'Jurassic Park (1993)',
 'Terminator 2: Judgment Day (1991)',
 'Total Recall (1990)',
 'Matrix, The (1999)',
 'Fifth Element, The (1997)',
 'Independence Day (ID4) (1996)',
 'Lost World: Jurassic Park, The (1997)',
 'Face/Off (1997)',
 'Rocky Horror Picture Show, The (1975)',
 'Schlafes Bruder (Brother of Sleep) (1995)',
 'Galaxy Quest (1999)',
 'Bob Roberts (1992)',
 'True Lies (1994)',
 'Braveheart (1995)']

In [25]:
def get_similar_movie(movie_title: str):
    movie_id = movie_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id, N=15)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [27]:
get_similar_movie('Star Wars: Episode IV - A New Hope (1977)')

['Star Wars: Episode IV - A New Hope (1977)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Raiders of the Lost Ark (1981)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Terminator, The (1984)',
 'Alien (1979)',
 'Matrix, The (1999)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Indiana Jones and the Last Crusade (1989)',
 'Back to the Future (1985)',
 'E.T. the Extra-Terrestrial (1982)',
 'Terminator 2: Judgment Day (1991)',
 'Aliens (1986)',
 'Blade Runner (1982)',
 'Jaws (1975)']

---

### 2.8 가장 좋아할 만한 영화 추천

In [28]:
user = user_to_idx['ko']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=15, filter_already_liked_items=True)
movie_recommended

[(92, 0.5666804),
 (117, 0.54229254),
 (64, 0.53713334),
 (107, 0.5250795),
 (120, 0.40227902),
 (60, 0.3796824),
 (50, 0.33797595),
 (200, 0.33239767),
 (26, 0.32249716),
 (62, 0.31308138),
 (5, 0.312742),
 (160, 0.29527792),
 (87, 0.2915262),
 (110, 0.29151115),
 (48, 0.2710328)]

In [29]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Terminator 2: Judgment Day (1991)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Jurassic Park (1993)',
 'Raiders of the Lost Ark (1981)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Toy Story 2 (1999)',
 'Terminator, The (1984)',
 'E.T. the Extra-Terrestrial (1982)',
 'Total Recall (1990)',
 'Princess Bride, The (1987)',
 'Forrest Gump (1994)',
 'Braveheart (1995)',
 'Groundhog Day (1993)',
 'Saving Private Ryan (1998)']

In [30]:
StarWars = movie_to_idx['Star Wars: Episode V - The Empire Strikes Back (1980)']
explain = als_model.explain(user, csr_data, itemid=StarWars)

In [31]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Star Wars: Episode IV - A New Hope (1977)', 0.3627000051949986),
 ('Back to the Future (1985)', 0.09040243436825258),
 ('Matrix, The (1999)', 0.05749216278212149),
 ('Men in Black (1997)', 0.015028301775183054),
 ('Toy Story (1995)', 0.0065962631161603415)]

---

## 3. 결론
### 3.1 회고

- 이번 노드는 개인적으로 재미가 붙었던 것 같다. 추천 시스템에 대해 자세히 다루어 볼 수 있어 흥미로웠기 때문이다. 다만 중간에.. 오류나 벡터 내적수치가 음수로 나와서 고생좀 했던 것 같다. 제목(title)에 데이터와 같은 제목으로 달아주지 않았기에 예상치 못했던 상황들도 생겼었는데, 년도를 포함한 풀네임으로 title을 작성했기 때문이었다. 또 아쉬웠던 점은 데이터셋에 최신 영화는 반영되지 않은 것 같았다. 내가 평소 선호하던 영화들을 기입하지 못했고, 대부분 20세기의 영화이기에 정보가 부족하여 제대로된 추천을 받았는지 알 수가 없어서 아쉬움이 크게 남는다.